In [1]:
using DataFrames, DataFramesMeta, Gadfly, Compose, XGBoost, Lazy

In [2]:
t = readtable("train.csv")
t[1:4, :]

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
1,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
2,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
3,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.42436302145,37.8004143219856
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.42699532676599,37.80087263276921


In [170]:
function fixData(df, includeLabel = false)
    fmt = "yyyy-mm-dd HH:MM:SS"
    df[:parsedDate] = DateTime(df[:Dates],fmt)
    df[:hour] = Dates.hour(df[:parsedDate])
    df[:minute] = Dates.minute(df[:parsedDate])
    df[:month] = Dates.month(df[:parsedDate])
    df[:day] = Dates.day(df[:parsedDate])
    df[:year] = Dates.year(df[:parsedDate])
    
    categoryCols = [
        (:Category, :category),
        (:DayOfWeek, :dayOfWeek), 
        (:PdDistrict, :pdDistrict)
    ]
    
    cols = [:year, :month, :day, :hour, :minute, :dayOfWeek, :pdDistrict, :X, :Y]
    labelCol = :category
    
    if (includeLabel)
        push!(cols, labelCol)
    end
    
    pool!(df)
    
    for c in categoryCols
        if c[1] in names(df)
           df[c[2]] = convert(Array{Int64,1},[getpoolidx(df[c[1]], v) for v in df[c[1]]]) 
        end
    end
            
    pool!(df)
    
    # Make sure label is 0 indexed.
    if (includeLabel)
        df[labelCol] = df[labelCol] - minimum(df[labelCol])
    end
    
    (df, @select(df, cols))
end

fixData (generic function with 2 methods)

In [163]:
(_, df) = fixData(t, true)

(878049×18 DataFrames.DataFrame
│ Row    │ Dates                 │ Category                 │
├────────┼───────────────────────┼──────────────────────────┤
│ 1      │ "2015-05-13 23:53:00" │ "WARRANTS"               │
│ 2      │ "2015-05-13 23:53:00" │ "OTHER OFFENSES"         │
│ 3      │ "2015-05-13 23:33:00" │ "OTHER OFFENSES"         │
│ 4      │ "2015-05-13 23:30:00" │ "LARCENY/THEFT"          │
│ 5      │ "2015-05-13 23:30:00" │ "LARCENY/THEFT"          │
│ 6      │ "2015-05-13 23:30:00" │ "LARCENY/THEFT"          │
│ 7      │ "2015-05-13 23:30:00" │ "VEHICLE THEFT"          │
│ 8      │ "2015-05-13 23:30:00" │ "VEHICLE THEFT"          │
│ 9      │ "2015-05-13 23:00:00" │ "LARCENY/THEFT"          │
│ 10     │ "2015-05-13 23:00:00" │ "LARCENY/THEFT"          │
│ 11     │ "2015-05-13 22:58:00" │ "LARCENY/THEFT"          │
⋮
│ 878038 │ "2003-01-06 00:55:00" │ "VANDALISM"              │
│ 878039 │ "2003-01-06 00:42:00" │ "WARRANTS"               │
│ 878040 │ "2003-01-06 00:40:00" │ "

In [171]:
(_, test) = fixData(readtable("test.csv"))

(884262×15 DataFrames.DataFrame
│ Row    │ Id     │ Dates                 │ DayOfWeek   │ PdDistrict  │
├────────┼────────┼───────────────────────┼─────────────┼─────────────┤
│ 1      │ 0      │ "2015-05-10 23:59:00" │ "Sunday"    │ "BAYVIEW"   │
│ 2      │ 1      │ "2015-05-10 23:51:00" │ "Sunday"    │ "BAYVIEW"   │
│ 3      │ 2      │ "2015-05-10 23:50:00" │ "Sunday"    │ "NORTHERN"  │
│ 4      │ 3      │ "2015-05-10 23:45:00" │ "Sunday"    │ "INGLESIDE" │
│ 5      │ 4      │ "2015-05-10 23:45:00" │ "Sunday"    │ "INGLESIDE" │
│ 6      │ 5      │ "2015-05-10 23:40:00" │ "Sunday"    │ "TARAVAL"   │
│ 7      │ 6      │ "2015-05-10 23:30:00" │ "Sunday"    │ "INGLESIDE" │
│ 8      │ 7      │ "2015-05-10 23:30:00" │ "Sunday"    │ "INGLESIDE" │
│ 9      │ 8      │ "2015-05-10 23:10:00" │ "Sunday"    │ "MISSION"   │
│ 10     │ 9      │ "2015-05-10 23:10:00" │ "Sunday"    │ "CENTRAL"   │
│ 11     │ 10     │ "2015-05-10 23:00:00" │ "Sunday"    │ "INGLESIDE" │
⋮
│ 884251 │ 884250 │ "2003-01-0

In [7]:
function splitrandom_indices(n::Integer, proportion::Real)
	indices = [1:n;]
	shuffle!(indices)
    splitindex = round(Integer, n * proportion)
	return indices[1:splitindex], indices[(splitindex + 1):n]
end

function splitrandom(df::AbstractDataFrame, proportion::Real)
    included, excluded = splitrandom_indices(nrow(df), proportion)
    return df[included, :], df[excluded, :]
end
# randomly split the data 80/20 for train and test


splitrandom (generic function with 1 method)

In [172]:
size(test)

(884262,9)

In [174]:
#train, validate = splitrandom(df, .8)
train = df
numCats = length(unique(train[:category]))
dtrain = DMatrix(convert(Array{Real},train), label = convert(Array, train[:category]))
#dvalidate = DMatrix(convert(Array{Real},validate), label = convert(Array, validate[:category]))
dtest = DMatrix(convert(Array{Real},test))

XGBoost.DMatrix(Ptr{Void} @0x00007fd53d7b4660,_setinfo)

In [175]:
# watchlist = [(dvalidate,"eval"), (dtrain,"train")]
watchlist = [(dtrain,"train")]

1-element Array{Tuple{XGBoost.DMatrix,ASCIIString},1}:
 (XGBoost.DMatrix(Ptr{Void} @0x00007fd53d621670,_setinfo),"train")

In [176]:
num_round = 9
bst = xgboost(dtrain, num_round, eta=0.5, max_depth=6, colsample_bytree=0.5, watchlist=watchlist, num_class=numCats, objective="multi:softprob")

[1]	train-merror:0.140346
[2]	train-merror:0.048513
[3]	train-merror:0.035618
[4]	train-merror:0.019281
[5]	train-merror:0.013741
[6]	train-merror:0.005204
[7]	train-merror:0.002810
[8]	train-merror:0.000819
[9]	train-merror:0.000130


XGBoost.Booster(Ptr{Void} @0x00007fd53963afd0)

In [177]:
labels = get_info(dtest, "label")
preds = reshape(XGBoost.predict(bst, dtest),  size(test)[1], numCats)
println(preds[1,:])

Float32[0.99828666 3.0388655e-5 7.5455937e-6 9.930596e-6 0.000943935 1.1674844e-5 1.0669014e-5 7.768442e-6 1.2200244e-5 7.916466e-6 7.561709e-6 9.505445e-6 8.703618e-6 0.99840957 5.272458e-5 7.5457906e-6 9.645555e-6 0.00042563883 9.254299e-6 9.666702e-6 7.755171e-6 1.2203283e-5 8.456398e-6 7.557396e-6 8.17794e-6 1.06168645e-5 0.9987068 3.714876e-5 7.547044e-6 9.72117e-6 0.0007122694 1.2572336e-5 1.0500411e-5 7.771735e-6 1.2198521e-5 8.332977e-6 7.5621197e-6 9.5660325e-6 9.240734e-6]


In [178]:
outdata = DataFrame(id = collect(0:size(test)[1]-1))
crimes = levels(t[:Category])
[outdata[Symbol(crimes[i])] = preds[:, i] for i in 1:numCats]
writetable("pred.csv", outdata)
outdata

,id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
1,0,0.99828666,3.0388655e-5,7.5455937e-6,9.930596e-6,0.000943935,1.1674844e-5,1.0669014e-5,7.768442e-6,1.2200244e-5,7.916466e-6,7.561709e-6,9.505445e-6,8.703618e-6,0.99840957,5.272458e-5,7.5457906e-6,9.645555e-6,0.00042563883,9.254299e-6,9.666702e-6,7.755171e-6,1.2203283e-5,8.456398e-6,7.557396e-6,8.17794e-6,1.06168645e-5,0.9987068,3.714876e-5,7.547044e-6,9.72117e-6,0.0007122694,1.2572336e-5,1.0500411e-5,7.771735e-6,1.2198521e-5,8.332977e-6,7.5621197e-6,9.5660325e-6,9.240734e-6
2,1,0.000110838824,1.4038452e-5,7.789684e-6,1.3506306e-5,7.5327425e-6,8.2527695e-6,1.0974457e-5,7.5855696e-6,1.17693935e-5,9.895937e-5,8.2475235e-6,8.504219e-6,7.833311e-6,0.00010761346,1.3919859e-5,7.707941e-6,0.003937592,7.5358266e-6,9.050042e-6,9.894177e-6,7.5726116e-6,1.1613203e-5,0.00013561008,8.24282e-6,7.961126e-6,7.835301e-6,0.00011063039,1.3990818e-5,7.709221e-6,4.2573207e-5,7.531398e-6,9.046313e-6,1.0046161e-5,7.588786e-6,1.1608671e-5,7.671616e-5,8.247972e-6,8.315696e-6,7.832852e-6
3,2,1.4482208e-5,7.672589e-6,8.4652565e-6,2.4797992e-5,7.222859e-6,8.411128e-6,7.5561265e-6,7.540189e-6,8.651203e-6,9.196662e-6,7.980939e-6,7.581524e-6,8.204348e-6,1.44839905e-5,7.674216e-6,8.465478e-6,2.1031674e-5,7.252582e-6,1.2876818e-5,7.5598327e-6,7.5273088e-6,8.653358e-6,8.23862e-6,8.12394e-6,7.5803996e-6,8.1694425e-6,1.4575252e-5,7.672889e-6,8.493457e-6,2.1043392e-5,7.289156e-6,9.116334e-6,7.5543103e-6,7.543386e-6,8.649981e-6,8.51867e-6,7.981373e-6,7.638461e-6,8.066555e-6
4,3,1.2197987e-5,7.966279e-6,7.561983e-6,1.1549538e-5,8.525643e-6,0.9960806,2.8624012e-5,7.541303e-6,9.9626395e-6,0.0008556791,9.849769e-6,1.0826145e-5,7.772537e-6,1.2199488e-5,7.790468e-6,7.56218e-6,8.8481775e-6,7.703682e-6,0.9986934,1.6898563e-5,7.528421e-6,9.8408855e-6,0.00038506417,1.0593261e-5,1.0212632e-5,7.774512e-6,1.2203121e-5,7.669685e-6,7.563436e-6,1.0405607e-5,8.497759e-6,0.99841285,1.788529e-5,7.5445005e-6,9.759068e-6,0.0010627704,1.1861289e-5,1.0425316e-5,7.772081e-6
5,4,1.2643201e-5,0.00026940208,8.247823e-6,8.594576e-6,7.832469e-6,0.00010804187,1.06402795e-5,7.708127e-6,2.5918422e-5,7.533441e-6,8.901242e-6,1.0529358e-5,7.589569e-6,1.1609592e-5,0.00028178497,8.248037e-6,8.112138e-6,7.835676e-6,0.000106524785,1.0009146e-5,7.690198e-6,2.4830608e-5,7.536074e-6,8.896165e-6,1.0450531e-5,7.591497e-6,1.1797581e-5,0.000500725,8.249407e-6,8.247945e-6,7.831071e-6,0.00010853679,1.0395848e-5,7.706623e-6,2.526043e-5,7.531866e-6,8.901725e-6,9.31065e-6,7.589124e-6
6,5,8.649603e-6,9.3471135e-6,7.896035e-6,7.5776297e-6,8.330944e-6,1.8781166e-5,7.6721835e-6,8.4604435e-6,1.3078432e-5,7.0612173e-6,8.864054e-6,7.5590424e-6,7.5441644e-6,9.051048e-6,8.861682e-6,7.89624e-6,7.552436e-6,9.008973e-6,1.4488109e-5,7.675947e-6,8.1231265e-6,4.740733e-5,7.0250567e-6,9.115673e-6,7.5579214e-6,7.5460807e-6,8.653243e-6,8.860149e-6,8.799078e-6,7.57768e-6,1.4279221e-5,1.4484039e-5,7.670339e-6,8.140477e-6,5.053476e-5,7.1545965e-6,8.864535e-6,7.557762e-6,7.543722e-6
7,6,9.960798e-6,0.0007227093,8.464835e-6,1.4173601e-5,8.005665e-6,1.2171031e-5,8.436151e-6,7.557683e-6,1.00018615e-5,9.469042e-6,0.99870586,1.76346e-5,7.5452795e-6,9.730288e-6,0.0004889085,8.171786e-6,8.816129e-6,7.774884e-6,1.2202957e-5,8.233298e-6,7.544773e-6,8.144008e-6,7.8744915e-6,0.9987418,1.610944e-5,7.5471958e-6,9.837348e-6,0.00045110917,8.623328e-6,1.0886851e-5,8.0042355e-6,1.2199529e-5,9.728598e-6,7.5608873e-6,9.990803e-6,8.174789e-6,0.99834704,2.594996e-5,7.5448365e-6
8,7,1.4581673e-5,7.532983e-6,8.901564e-6,1.5938862e-5,7.5887533e-6,

In [179]:
size(outdata)

(884262,40)